# Titanic: Survival Model

Build and train a model to predict survival on the Titanic using a [cleaned and split dataset](https://huggingface.co/datasets/jamieoliver/titanic-2410), and upload the model to Hugging Face.

Based on https://github.com/fastai/course22/blob/master/clean/05-linear-model-and-neural-net-from-scratch.ipynb using the dataset from https://www.kaggle.com/competitions/titanic.

Plan
- [x] Download [cleaned and split dataset](https://huggingface.co/datasets/jamieoliver/titanic-2410) from Hugging Face
- [x] Prepare data for model
    - [x] Load training dataset as PyTorch tensors
    - [x] Normalise training dataset
- [x] Train linear model
    - [x] Set up coefficients
    - [x] Set up gradient descent step
    - [x] Run training loop
- [x] Train neural network
    - [x] Set up coefficients
    - [x] Run training loop
- [x] Train deep neural network
    - [x] Set up coefficients
    - [x] Run training loop
- [x] Recreate as PyTorch module
- [ ] Test model
- [ ] Upload model to Hugging Face

##  Download Dataset from Hugging Face

In [1]:
from datasets import *

datasetDict = load_dataset('jamieoliver/titanic-2410', revision='1.0')
datasetDict

DatasetDict({
    train: Dataset({
        features: ['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
        num_rows: 712
    })
    validation: Dataset({
        features: ['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
        num_rows: 179
    })
    test: Dataset({
        features: ['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
        num_rows: 418
    })
})

## Prepare Data for Model

### Load Training Dataset as PyTorch Tensors

In [2]:
import torch
from torch import tensor

torch.set_default_device('cuda')
torch.set_printoptions(linewidth=120, edgeitems=10)

In [3]:
train_dataset = datasetDict['train']
train_dataset

Dataset({
    features: ['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
    num_rows: 712
})

The dependent variable is the variable we are predicting i.e. `survived`.

In [4]:
dependent_var = tensor(train_dataset.to_pandas()['Survived'].values, dtype=torch.float)
dependent_var

tensor([0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1.,
        0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
        1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0.,
        1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 1., 1., 0., 1., 0., 0., 0., 

In [5]:
dependent_var.shape

torch.Size([712])

The independent variables are the variables we will use to make the prediction. Note that we use a trick in mutiplying the Pandas DataFrame by 1 to convert booleans to integers.

In [6]:
independent_cols = ['Age', 'SibSp', 'Parch', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
                    'Embarked_C', 'Embarked_Q', 'Embarked_S']

independent_vars = tensor((train_dataset.to_pandas()*1)[independent_cols].values, dtype=torch.float)
independent_vars

tensor([[22.0000,  0.0000,  0.0000,  2.0949,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000],
        [24.0000,  2.0000,  0.0000,  4.3108,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000],
        [24.0000,  0.0000,  0.0000,  5.4316,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,  0.0000,  0.0000],
        [32.0000,  0.0000,  0.0000,  4.3476,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000],
        [30.0000,  0.0000,  0.0000,  2.1102,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000],
        [18.0000,  0.0000,  2.0000,  2.6391,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [47.0000,  1.0000,  1.0000,  3.9807,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [34.0000,  0.0000,  0.0000,  2.4423,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [39.0000,  1.0000,  1.0000,  4.7175,  1.0000,  0

In [7]:
independent_vars.shape

torch.Size([712, 12])

### Normalise Training Dataset

In [8]:
max_vals,indices = independent_vars.max(dim=0)
independent_vars /= max_vals
independent_vars

tensor([[0.2973, 0.0000, 0.0000, 0.3357, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.3243, 0.2500, 0.0000, 0.6907, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.3243, 0.0000, 0.0000, 0.8703, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000],
        [0.4324, 0.0000, 0.0000, 0.6966, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000],
        [0.4054, 0.0000, 0.0000, 0.3381, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.2432, 0.0000, 0.3333, 0.4229, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.6351, 0.1250, 0.1667, 0.6378, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.4595, 0.0000, 0.0000, 0.3913, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.5270, 0.1250, 0.1667, 0.7559, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000],
        [0.6757, 0.0000, 0.1667, 0.8838, 1.000

## Train Linear Model

### Set Up Coefficients

Initialise random coefficients as a column vector.

In [9]:
num_coeffs = independent_vars.shape[1]
torch.manual_seed(42)
coeffs = torch.rand(num_coeffs, 1) - 0.5
coeffs

tensor([[ 0.1130],
        [-0.4899],
        [-0.1016],
        [-0.4597],
        [-0.3437],
        [-0.0175],
        [ 0.2362],
        [-0.0940],
        [ 0.0189],
        [-0.2133],
        [-0.2584],
        [ 0.4228]], device='cuda:0')

Transpose the dependent variable into a column vector.

In [10]:
dependent_var = dependent_var[:,None]
dependent_var[:10]

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')

In [11]:
predictions = independent_vars@coeffs
predictions[:10]

tensor([[ 0.5573],
        [ 0.0208],
        [-0.9016],
        [-0.9224],
        [ 0.5683],
        [ 0.1105],
        [-0.3145],
        [ 0.1833],
        [-1.0172],
        [-0.9979]], device='cuda:0')

In [12]:
loss = torch.abs(predictions - dependent_var).mean()
loss

tensor(0.6676, device='cuda:0')

In [13]:
def calc_predictions(coeffs, independent_vars):
    return torch.sigmoid(independent_vars@coeffs).sum(axis=1)

def calc_loss(coeffs, independent_vars, dependent_var):
    return torch.abs(calc_predictions(coeffs, independent_vars) - dependent_var).mean()
    

### Set Up Gradient Descent Step

In [14]:
coeffs.requires_grad_()

tensor([[ 0.1130],
        [-0.4899],
        [-0.1016],
        [-0.4597],
        [-0.3437],
        [-0.0175],
        [ 0.2362],
        [-0.0940],
        [ 0.0189],
        [-0.2133],
        [-0.2584],
        [ 0.4228]], device='cuda:0', requires_grad=True)

In [15]:
loss = calc_loss(coeffs, independent_vars, dependent_var)
loss

tensor(0.5034, device='cuda:0', grad_fn=<MeanBackward0>)

In [16]:
loss.backward()
coeffs.grad

tensor([[0.0239],
        [0.0041],
        [0.0039],
        [0.0296],
        [0.0143],
        [0.0138],
        [0.0348],
        [0.0213],
        [0.0416],
        [0.0118],
        [0.0058],
        [0.0453]], device='cuda:0')

In [17]:
loss = calc_loss(coeffs, independent_vars, dependent_var)
loss.backward()
with torch.no_grad():
    coeffs.sub_(coeffs.grad * 0.1)
    coeffs.grad.zero_()
    print(calc_loss(coeffs, independent_vars, dependent_var))

tensor(0.5019, device='cuda:0')


### Run Training Loop

In [18]:
def update_coeffs(coeffs, learning_rate):
    coeffs.sub_(coeffs.grad * learning_rate)
    coeffs.grad.zero_()

In [19]:
def one_epoch(coeffs, learning_rate):
    loss = calc_loss(coeffs, independent_vars, dependent_var)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, learning_rate)
    print(f'{loss:.3f}', end='; ')

In [20]:
def init_coeffs():
    return (torch.rand(num_coeffs, 1) - 0.5).requires_grad_()

In [21]:
def train_model(epochs=30, learning_rate=0.01):
    torch.manual_seed(442)
    coeffs = init_coeffs()
    for i in range (epochs):
        one_epoch(coeffs, learning_rate)
        
    return coeffs

In [22]:
coeffs = train_model(epochs=20, learning_rate=100)

0.517; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 0.368; 

In [23]:
def show_coeffs():
    return dict(zip(independent_cols, coeffs.requires_grad_(False)))

show_coeffs()

{'Age': tensor([-2.0584], device='cuda:0'),
 'SibSp': tensor([-0.8960], device='cuda:0'),
 'Parch': tensor([-0.2124], device='cuda:0'),
 'LogFare': tensor([-2.8352], device='cuda:0'),
 'Pclass_1': tensor([-1.3458], device='cuda:0'),
 'Pclass_2': tensor([-1.7492], device='cuda:0'),
 'Pclass_3': tensor([-3.9983], device='cuda:0'),
 'Sex_female': tensor([-2.1495], device='cuda:0'),
 'Sex_male': tensor([-3.9429], device='cuda:0'),
 'Embarked_C': tensor([-0.9537], device='cuda:0'),
 'Embarked_Q': tensor([-0.3663], device='cuda:0'),
 'Embarked_S': tensor([-4.4945], device='cuda:0')}

## Train Neural Network

### Set up Coefficients

Initialisation of the coefficients for the neural network is similar to the linear model. However, we need to initialise the coefficients for each layer in the network. We do this by creating a list of tensors, one for each layer. The number of columns in the first layer is equal to the number of independent variables. The number of columns in each subsequent layer is equal to the number of hidden coefficients, which we set to a constant value. The number of columns in the last layer is equal to the number of dependent variables i.e. 1 in our case.

We divide the number of hidden coefficients by the number of columns in the first layer to ensure that the weights are distributed evenly across each layer. This ensures that the gradients for each layer will be similar and have a similar impact on the model's performance.

We add a constant to the last layer of coefficients so that the model can learn a bias term.

In [24]:
def init_coeffs(num_hidden_coeffs=20):
    layer_1 = (torch.rand(num_coeffs, num_hidden_coeffs) - 0.5) / num_hidden_coeffs
    layer_2 = torch.rand(num_hidden_coeffs, 1) - 0.3
    const = torch.rand(1)[0]
    return layer_1.requires_grad_(), layer_2.requires_grad_(), const.requires_grad_()

Calculation of predictions is also similar to the linear model except that we take the outputs from the first layer and pass them to the second layer. Note the use of `relu` to rectify the outputs of the first layer to ensure that they are positive.

In [25]:
import torch.nn.functional as F

def calc_predictions(coeffs, independent_vars):
  layer_1, layer_2, const = coeffs
  result = F.relu(independent_vars@layer_1)
  result = result@layer_2 + const
  return torch.sigmoid(result)
     

In [26]:
def update_coeffs(coeffs, learning_rate):
  for layer_coeffs in coeffs:
    layer_coeffs.sub_(layer_coeffs.grad * learning_rate)
    layer_coeffs.grad.zero_()

### Run Training Loop

In [27]:
coeffs = train_model(epochs=100, learning_rate=20)

0.543; 0.417; 0.271; 0.392; 0.226; 0.195; 0.188; 0.186; 0.186; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 

## Train Deep Neural Network

### Set up Coefficients

We now genericise the number of hidden layers and their sizes. We also add a constant term to each layer.

In [28]:
def init_coeffs(num_hidden_coeffs=[10, 10]):
  layer_sizes = [num_coeffs] + num_hidden_coeffs + [1]
  num_layers = len(layer_sizes)

  layers = [(torch.rand(layer_sizes[i], layer_sizes[i+1]) - 0.5) / layer_sizes[i + 1] * 4 for i in range(num_layers - 1)]
  consts = [(torch.rand(1)[0] - 0.5) * 0.1 for i in range(num_layers - 1)]

  for layer in layers + consts:
    layer.requires_grad_()

  return layers, consts

Calculation of predictions proceeds largely as before but instead we loop over each layer rather than explicitly writing them out.

In [29]:
def calc_predictions(coeffs, independent_vars):
  layers, consts = coeffs
  num_layers = len(layers)
  result = independent_vars
  
  for i, layer in enumerate(layers):
    result = result@layer + consts[i]
    if i != num_layers - 1:
      result = F.relu(result)

  return torch.sigmoid(result)

A minor change to to the training loop is required as we now have a list of coefficients rather than a single set.

In [30]:
def update_coeffs(coeffs, learning_rate):
  layers, consts = coeffs
  for layer in layers + consts:
    layer.sub_(layer.grad * learning_rate)
    layer.grad.zero_()

### Run Training Loop

In [31]:
coeffs = train_model(epochs=100, learning_rate=4)

0.505; 0.477; 0.403; 0.367; 0.367; 0.366; 0.365; 0.361; 0.349; 0.339; 0.338; 0.332; 0.310; 0.294; 0.254; 0.216; 0.204; 0.208; 0.199; 0.212; 0.193; 0.195; 0.205; 0.189; 0.188; 0.188; 0.187; 0.187; 0.187; 0.186; 0.186; 0.186; 0.186; 0.186; 0.186; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.185; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.184; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 0.183; 

## Recreate as PyTorch Module

We subclass the PyTorch Sequential class to create a model that we can train. The layers from the previous example are recreated as members of the class, and are initialised with random weights as before.

In [32]:
import torch.nn as nn

class TitanicModel(nn.Sequential):
  def __init__(self):
    super(TitanicModel, self).__init__(nn.Linear(num_coeffs, 10),
                                       nn.ReLU(),
                                       nn.Linear(10, 10),
                                       nn.ReLU(),
                                       nn.Linear(10, 1),
                                       nn.Sigmoid())
    
    self.apply(self._init_weights)
    
  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      module.weight.data.normal_(mean=0.0, std=0.5)

The training loop is similar to before with the exception that we use the PyTorch optimisation machinery rather than creating our own.  

In [33]:
import torch.optim as optim

def train_model(epochs=30, learning_rate=10):
  torch.manual_seed(442)
  model = TitanicModel()
  calc_loss = nn.L1Loss()
  optimizer = optim.SGD(model.parameters(), lr=learning_rate)

  for epoch in range(epochs):
    optimizer.zero_grad()

    output = model(independent_vars)
    loss = calc_loss(output, dependent_var)

    loss.backward()
    optimizer.step()
    print(f'{loss:.3f}', end='; ')

  return model

Training progresses similar to before:

In [34]:
train_model(epochs=25, learning_rate=12.5)

0.480; 0.349; 0.272; 0.320; 0.334; 0.278; 0.212; 0.209; 0.208; 0.207; 0.202; 0.198; 0.198; 0.200; 0.209; 0.204; 0.189; 0.187; 0.186; 0.186; 0.186; 0.184; 0.184; 0.184; 0.183; 

TitanicModel(
  (0): Linear(in_features=12, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=1, bias=True)
  (5): Sigmoid()
)